In [1]:
# from earthdata import Auth, DataCollections, DataGranules, Store # Used to ingest data from NSIDC
# import geopandas as gpd # Used to subset regions using shapefile
# import rioxarray as rx # Used to read raster data from hdf5 files


In [ ]:
#COMMENTED OUT SECTIONS WERE PULLED FOR TESTING. ALL THAT IS NEEDED IS THE BOTTOM CELL

In [1]:
# Set working directories

data_loc = "/home/jovyan/data/"

In [2]:
# # Authenticate Earthdata credentials

# auth = Auth().login(strategy='netrc')
# if auth.authenticated is False:
#     auth = Auth().login(strategy='interactive')

In [3]:
# bd = gpd.read_file(data_loc+"/boundaries/IceBoundaries_Antarctica_v02.shp")
# # Plot shapefile to make sure it as expected
# bd.plot()

# # 'Geometry' can be extracted - here, we print out the head of the geometry field
# print(bd['geometry'].head())

In [4]:
# # Isolated only grounded ice (GR) for the Totten basin
# tot = bd[(bd["NAME"] == "Totten") & (bd["TYPE"] == "GR")]

# # Plot the Totten subset
# tot.plot()

# # View tot to make sure it is as expected
# tot

In [5]:
# ## Clip Tot basin bedrock topography: use drainage basin outline shapefile to clip the bedmachine gridded/raster file

# ds=rx.open_rasterio(data_loc+"BedMachineAntarctica_2019-11-05_v01.nc", variable = "bed")

# # Double check the crs of BedMachineAntarctica and drainage basin shapefile before clipping
# print(f"BedMachineAntarctica CRS: ", ds.bed.rio.crs)
# print(f"Tot shapefile CRS: ", tot.crs)

# ds.rio.set_spatial_dims(x_dim="x", y_dim="y", inplace=True)
# ds.rio.write_crs("EPSG:3031", inplace=True)
# Bed_tot = ds.rio.clip(tot.geometry, tot.crs)

# Bed_tot_masked = Bed_tot.where(Bed_tot['bed'] != -9999.) # Replace -9999. with nan
# Bed_tot_masked['bed'].plot()

# ## Save a local copy as a raster or netcdf
# Bed_tot.bed.rio.to_raster(data_loc+'Tot_BedMachineAntarctica.tif')
# # Bed_tot.to_netcdf(data_loc+'Tot_BedMachineAntarctica.nc')

In [8]:
print(f"Resolution of bed_tot: ", Bed_tot.rio.resolution())

Resolution of bed_tot:  (500.0, -500.0)


In [9]:
#Resample clipped data to matching resolutions

from osgeo import gdal

# REFRENCE FILE METHOD: choose highest resolution data product and import resolution
#ref = gdal.Open(data_loc+"/its_live/ANT_G0240_0000.nc", 'r')
#perhaps ATL 14 will provide a better ref resolution. gdal != .nc for this transform

a=240    #its-live resolution =240m
x_res = a           #refTrans[1] with gdal
y_res = -a          #-refTrans[5] with gdal #y values need a "-" as they are stored negative

inputFile =data_loc+"Tot_BedMachineAntarctica.tif"
outputFile = data_loc+"Tot_Resample_BedMachineAntarctica.tif"

#call gdal.Warp
kwargs = {"format": "GTiff", "xRes": x_res, "yRes": y_res}
ds = gdal.Warp(outputFile, inputFile, **kwargs)

#test new resolution
test=rx.open_rasterio(data_loc+"Tot_Resample_BedMachineAntarctica.tif")
print(f"New Resolution: ", test.rio.resolution())


new resolution:  (240.0, -240.0)
